In [2]:
!pip install requests



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

ModuleNotFoundError: No module named 'bs4'

In [226]:
# requesting the faculty page
URL_lhr = "http://lhr.nu.edu.pk/faculty/"
r_lhr = requests.get(URL_lhr) 
print(r_lhr) 

<Response [200]>


In [227]:
# creating a beautiful soup object
soup_lhr = BeautifulSoup(r_lhr.content, 'html.parser') 

In [228]:
# Extracting the faculty member cards
faculty_lhr = soup_lhr.findAll('div', class_='facultyCard')


In [229]:
ID=[]
name=[]
email=[]
designation=[]
img=[]
highest_education=[]
HEC_approved=[]
dept=[]
extension=[]


In [230]:
# extracting ID , image , name , email and designation along with whether or not the faculty member is an HEC Approved Supervisor  from the faculty page
for f in faculty_lhr:
    
    try:
        if(f.find('p',class_='small').text.strip().split('\n')[2]):
            HEC_approved.append(True)
            
    except IndexError:
        HEC_approved.append(False)
    
    try:
        ID.append(int(f.find('a',class_='faculty-link')['href'].split('/')[-1]))
        
    except:
        ID.append(-1)
        
    name.append(f.find('h5',class_='text-center').text)
        
    designation.append(f.find('p',class_='small').text.strip().split('\n')[0])
    
    email.append(f.find('p',class_='mb-0').text)
    
    img.append("https://lhr.nu.edu.pk" + f.find('img', class_="card-img-top")['src'])
    
    

Extracting the dept for each faculty member

In [232]:
computer_dept = soup_lhr.find('div',id='fsc')
computer_dept_faculty = computer_dept.descendants


In [233]:
for element in computer_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Computing')


In [234]:
ee_dept = soup_lhr.find('div',id='ee')
ee_dept_faculty = ee_dept.descendants


In [235]:
for element in ee_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Electrical Engineering')
   

In [236]:
cv_dept = soup_lhr.find('div',id='cv')
cv_dept_faculty = cv_dept.descendants


In [237]:
for element in cv_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Civil Engineering')


In [238]:
fsm_dept = soup_lhr.find('div',id='fsm')
fsm_dept_faculty = fsm_dept.descendants

In [239]:
for element in fsm_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Management')


In [240]:
ss_dept = soup_lhr.find('div',id='ss')
ss_dept_faculty = ss_dept.descendants


In [241]:
for element in ss_dept_faculty:
    if element.name == 'h5' and 'text-center' in element.get('class', []):
            dept.append('Sciences and Humanities')


175


In [242]:
# extracting the links for each faculty member's profile and then extracting the highest education and extension  

for f in faculty_lhr:
    
    profile_URL = "https://lhr.nu.edu.pk" + f.find('a',class_='faculty-link')['href']
    profile_page = requests.get(profile_URL)
    profile_content = BeautifulSoup(profile_page.content, 'html.parser')
    profile = profile_content.find('div',class_='container')
    
    try:
        highest_education.append(profile.find('li').text)
        
    except AttributeError:
            highest_education.append('Not Available')
          
    try:
        extension.append(int(profile.find('span',class_="small").text.split()[-1][-3:]))
    
    except ValueError:
        extension.append(-1)

In [245]:
# creating the dataframe
lhr_df = pd.DataFrame({'ID': ID , 'Name': name , 'Designation': designation , 'HEC Approved PHD Supervisor': HEC_approved ,
                       'Highest Education': highest_education , 'Email' : email , 'Department': dept , 'Extension': extension ,
                       'ImageURL': img })
print(lhr_df)

lhr_df.to_csv('lhr.csv')

       ID                      Name          Designation  \
0    1238          Dr. Kashif Zafar      Professor & HOD   
1    4391  Dr. Asif Mahmood Gillani            Professor   
2    5424         Dr. Hammad Naveed            Professor   
3    4027            Dr. Aamir Wali  Associate Professor   
4    6174            Dr. Arshad Ali  Associate Professor   
..    ...                       ...                  ...   
170  6420         Ms. Nokhaiz Zahra           Instructor   
171  6181            Ms. Rida Ahmed           Instructor   
172  6555              Ms. Aqsa Naz   Lecturer (English)   
173    -1    Mr. Hafiz Umair Gulzar  Lecturer (Islamiat)   
174  6548          Ms. Rubab Arshad   Lecturer (English)   

     HEC Approved PHD Supervisor  \
0                           True   
1                           True   
2                           True   
3                           True   
4                           True   
..                           ...   
170                        